# fMRI BOLD analysis

New default notebook for all single scan analyses, for both fMRI-only and simultaneous calcium/fMRI.

**Preparation for calcium/fMRI:**

- Copy the *regressorX_married.1D* files in the corresponding raw scan folder (e.g. subjectx/8/,  subjectx/9/  etc.). Will be recognized automatically.

- If no .1D file is present, only stim-evoked GLM is performed.

In [1]:
import os, sys, pathlib, shutil, glob
mymodules = os.fspath(pathlib.Path.cwd().joinpath('../CustomModules'))
if mymodules not in sys.path:
    sys.path.append(mymodules)
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import importlib
import helperfunctionsfMRI as fh
import MRIhelperfunctions as h
import calciumGroupFunctions as g

fh = importlib.reload(fh)
h = importlib.reload(h)
g = importlib.reload(g)

# Settings

In [2]:
paradigm = {}
folders = {}
averaging = {}

paradigm['TR'] = 1 # in seconds

paradigm['initial_cutoff'] = 30 # number of volumes to remove (in absence of dummy scans).

blurFWHM = 0.3 # In [mm]. Rule-of-thumb: 1.5x the voxel resolution.

highpass = 240 # period [s]. Any signal slower than this will be removed. Should be at least 2x inter-stim. interval


# --- For averaging of blocks:

averaging['baseline'] = 20 # baseline to consider before each block

averaging['time'] = 60 # how many seconds to display after end of stim_block


#======================================================================
folders['animal'] = '/media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13'

folders['scanNumber'] = '13'
#======================================================================


# --- EPI Template for coregistration ---
folders['template'] = '/media/aic/2/MRIDATA/Template7T/TEMPLATE.nii'

#Use this scan as new Template (warning, will override old template):
createNewTemplate = False


# --- Optional: ROI file:
folders['rois'] = '/media/aic/2/MRIDATA/Template7T/ROI3.nii'



# --- Define the stimulation paradigm (all units in seconds):
paradigm['baseline'] = 120
paradigm['stim_duration'] = 8
paradigm['Nblocks'] = 10
paradigm['ISI'] = 90    #inter-stimulus interval (on+off period)
paradigm['frequency'] = 4


paradigm['stim_times'], paradigm['stim_onsets'], paradigm['stim_times_all'], paradigm['singleblock'] = fh.CreateStimParadigm(paradigm)

stimulus times: [120, 210, 300, 390, 480, 570, 660, 750, 840, 930]


In [3]:
# this creates also the 'analysis' and 'scan' folders
folders = h.check_and_convert_single(folders, createNewTemplate)

Bru2 -a -o /media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13/analysis/13 /media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13/13/subject
3dTcat -overwrite -prefix /media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13/analysis/13/single_timepoint.nii /media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13/analysis/13/X13P1.nii'[0]'


CalledProcessError: Command '3dTcat -overwrite -prefix /media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13/analysis/13/single_timepoint.nii /media/aic/2/MRIDATA/Calciumanalysis/IsovsKetxyl_study/R_ketxyl/4Hz/6847_E13/analysis/13/X13P1.nii'[0]'' returned non-zero exit status 1.

# Data Preprocessing

In [ ]:
folders['cut'] = fh.initial_cutoff(folders['raw'], paradigm['initial_cutoff'])

In [ ]:
# takes several minutes
folders['coreg'] = h.coreg_epi_to_template(folders['template'], folders['cut'])

### Check for motion

In [ ]:
frame = pd.read_csv(os.path.join(folders['scan'],'dfile.1D'), sep=r'\s{2,}', engine='python')

%matplotlib qt
# frame.plot(subplots=True, layout=(9,1))
frame.iloc[:,1:7].plot(subplots=True, layout=(6,1))

High-pass filter, smoothing

In [ ]:
folders['cleaned'] = fh.filter_blur(folders['coreg'], highpass, blurFWHM)

# GLM Analyses

In [ ]:
# 1. Standard, using only stim. paradigm
fh.Deconvolve(folders['cleaned'],paradigm) # standard stim_evoked


# 2. Check if regressors present, use both in GLM f-test.
calcium_regressors = g.get_filepaths(folders['raw_folder'], fileending='*married.1D', subfolders=False)
if calcium_regressors:
    fh.Deconvolve_calcium_2ch(folders['cleaned'], paradigm, calcium_regressors)

# ROI Analysis

Can be run separately on previously analyzed datasets, but first execute all cells under "Settings" above

In [ ]:
folders['cleaned'] = fh.find_cleaned_data(folders)
folders['roidata'] = h.extract_roi(folders['cleaned'],folders['rois'])

roi_df = pd.read_csv(folders['roidata'], sep='\t')
del roi_df['File'], roi_df['Sub-brick']

Plotting whole time course

In [ ]:
%matplotlib qt

plt.figure()
for rois in roi_df.columns:
    plt.plot(roi_df[rois])
_ = [plt.axvline(_p, alpha=0.05, color='red') for _p in paradigm['stim_times_all']]
_ = plt.legend(roi_df.columns)

Plotting averages across stimulus blocks

In [ ]:
%matplotlib qt

roimeans = fh.averageblocks(roi_df, paradigm, averaging)

plt.figure()
_ = [plt.axvline(_p, alpha=0.05, color='red') for _p in paradigm['singleblock']]
plt.axhline(0,color='k')
plt.plot(roimeans)
_ = plt.legend(roi_df.columns)